In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

!unzip -uq "/content/drive/MyDrive/Research and Project Stuffs/BdSL/Datasets/dataset3.zip" -d "/content/drive/MyDrive/Research and Project Stuffs/BdSL/Datasets/"

In [ ]:
# example of progressively loading images from file
from keras.preprocessing.image import ImageDataGenerator
# create generator
datagen = ImageDataGenerator(validation_split=0.2)
img_height = 224
img_width = 224 
batch_size = 380
# prepare an iterators for each dataset
train_it = datagen.flow_from_directory('/content/drive/MyDrive/Research and Project Stuffs/BdSL/Datasets/dataset3/dataset',target_size=(img_height, img_width),batch_size = batch_size, subset='training')
val_it = datagen.flow_from_directory('/content/drive/MyDrive/Research and Project Stuffs/BdSL/Datasets/dataset3/dataset',target_size=(img_height, img_width), batch_size = batch_size, subset='validation')

Found 1523 images belonging to 38 classes.
Found 380 images belonging to 38 classes.


In [ ]:
from tensorflow.keras.applications import vgg19
# Init the VGG model
vgg_conv = vgg19.VGG19(weights='imagenet', include_top=False, input_shape=(img_height, img_height, 3))

80142336/80134624 [==============================] - 0s 0us/step


In [ ]:
# Freeze all the layers
for layer in vgg_conv.layers[:]:
    layer.trainable = False
# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7fd2d9b56750> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fd31f5eeb90> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fd2daad9210> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fd2daad96d0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fd2d9e25e10> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fd2d9e27bd0> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fd2d9e2e7d0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fd2d9e33e90> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fd2daad9110> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fd2dab744d0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fd2d9e44050> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 

In [ ]:
from keras.models import Sequential, Model
#from tensorflow.keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Activation, Embedding, Dense, Dropout, Flatten, Input,GlobalAveragePooling2D
from keras import activations
#from tensorflow.python.keras.layers.core import Dense, Dropout, Flatten
#from tensorflow.python.keras.layers import Input
from keras.optimizers import Adam, SGD
from keras.layers import LSTM
#from keras.layers import Dropout
#from keras.applications.vgg16 import VGG16
#from tensorflow.keras.applications import vgg16
from skimage.color import gray2rgb
#from tensorflow.keras.applications.inception_v3 import InceptionV3 
#from tensorflow.keras.applications import DenseNet201

In [ ]:
# Create the model
model = Sequential()

# Add the vgg convolutional base model
model.add(vgg_conv)

# Add new layers
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(38, activation='softmax'))

# Show a summary of the model. Check the number of trainable parameters
#model.summary()

In [ ]:
train_it.samples

1523

In [ ]:
import keras
#sgd = keras.optimizers.SGD(lr=0.00001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=['acc'])

In [ ]:
from sklearn.model_selection import KFold
import numpy as np

In [ ]:
num_folds = 10

In [ ]:
acc_per_fold = []
loss_per_fold = []

In [ ]:
y_img_batch, y_class_batch = train_it[0]

In [ ]:
nb_epochs = 100
batch_size = 380
history = model.fit_generator(
    train_it,
    steps_per_epoch = train_it.samples // batch_size,
    validation_data = val_it, 
    validation_steps = val_it.samples // batch_size,
    epochs = nb_epochs)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
4/4 [==============================] - 224s 28s/step - loss: 20.3033 - acc: 0.1155 - val_loss: 2.3290 - val_acc: 0.7447
Epoch 2/100
4/4 [==============================] - 10s 3s/step - loss: 9.5714 - acc: 0.4477 - val_loss: 0.3669 - val_acc: 0.9474
Epoch 3/100
4/4 [==============================] - 11s 3s/step - loss: 1.3457 - acc: 0.8233 - val_loss: 0.1173 - val_acc: 0.9868
Epoch 4/100
4/4 [==============================] - 12s 3s/step - loss: 0.6158 - acc: 0.9128 - val_loss: 0.0729 - val_acc: 0.9947
Epoch 5/100
4/4 [==============================] - 12s 3s/step - loss: 0.3763 - acc: 0.9328 - val_loss: 0.0495 - val_acc: 0.9947
Epoch 6/100
4/4 [==============================] - 12s 3s/step - loss: 0.4242 - acc: 0.9293 - val_loss: 0.0294 - val_acc: 0.9921
Epoch 7/100
4/4 [==============================] - 11s 4s/step - loss: 0.2678 - acc: 0.9511 - val_loss: 0.0410 - val_acc: 0.9947
Epoch 8/100
4/4 [==============================] - 11s 3s/step - loss: 0.2181 - acc: 0.9611 - 

In [ ]:
model.save("/content/drive/MyDrive/Research and Project Stuffs/BdSL/Datasets/Dataset_3_VGG19")

In [ ]:
val_it.classes

array([ 0,  0,  0, ..., 36, 36, 36], dtype=int32)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
num_of_test_samples = val_it.samples
batch_size = 128
Y_pred = model.predict(val_it)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(val_it.classes, y_pred))
print('Classification Report')
target_names = ['1', '10', '11', '12', '13', '14', '15', '16', '17',  '18', '19', '2', '20', '21',  '22',  '23',  '24',  '25',  '26',  '27','28', '29', '3', '30', '31', '32', '33', '34','35', '36', '38', '4', '5', '6', '7', '8', '9']
print(classification_report(val_it.classes, y_pred, target_names=target_names))

Confusion Matrix
[[0 3 1 ... 1 1 3]
 [0 3 1 ... 4 2 4]
 [0 3 1 ... 2 2 2]
 ...
 [4 3 3 ... 2 2 0]
 [0 3 4 ... 2 4 1]
 [2 3 2 ... 0 3 2]]
Classification Report
              precision    recall  f1-score   support

           1       0.00      0.00      0.00        72
          10       0.04      0.04      0.04        72
          11       0.01      0.01      0.01        72
          12       0.03      0.03      0.03        72
          13       0.00      0.00      0.00        72
          14       0.03      0.03      0.03        72
          15       0.01      0.01      0.01        72
          16       0.03      0.03      0.03        72
          17       0.03      0.03      0.03        72
          18       0.06      0.06      0.06        72
          19       0.06      0.06      0.06        72
           2       0.07      0.07      0.07        72
          20       0.04      0.04      0.04        72
          21       0.03      0.03      0.03        72
          22       0.06      0

In [ ]:
val_it.classes

array([ 0,  0,  0, ..., 36, 36, 36], dtype=int32)

In [ ]:
import keras
import numpy as np
model = keras.models.load_model("/content/drive/MyDrive/Research and Project Stuffs/BdSL/Datasets/Dataset_2_VGG19")

In [ ]:
y_img_batch, y_class_batch = val_it[0]
y_pred = np.argmax(model.predict(y_img_batch),-1)
y_true = np.argmax(y_class_batch,-1)
print(sum(y_pred==y_true)/batch_size)

0.9973684210526316


In [ ]:
verbosity = 1
import keras
from sklearn.model_selection import KFold
kfold = KFold(n_splits=num_folds, shuffle=True)

In [ ]:
no_epochs = 100

In [ ]:
# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(y_img_batch, y_class_batch):

  # Define the model architecture
  model = Sequential()

  # Add the vgg convolutional base model
  model.add(vgg_conv)

  # Add new layers
  model.add(Flatten())
  model.add(Dense(1024, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(38, activation='softmax'))

  # Compile the model
  model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=['acc'])


  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  # Fit data to model
  history = model.fit(y_img_batch[train], y_class_batch[train],
              batch_size=batch_size,
              epochs=no_epochs,
              verbose=verbosity)
  # Generate generalization metrics
  scores = model.evaluate(y_img_batch[test], y_class_batch[test], verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  # Increase fold number
  fold_no = fold_no + 1

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
1/1 [==============================] - 3s 3s/step - loss: 25.7293 - acc: 0.0322
Epoch 2/100
1/1 [==============================] - 2s 2s/step - loss: 10.7975 - acc: 0.2251
Epoch 3/100
1/1 [==============================] - 2s 2s/step - loss: 4.3525 - acc: 0.5029
Epoch 4/100
1/1 [==============================] - 2s 2s/step - loss: 2.0367 - acc: 0.6813
Epoch 5/100
1/1 [==============================] - 2s 2s/step - loss: 1.2323 - acc: 0.7982
Epoch 6/100
1/1 [==============================] - 2s 2s/step - loss: 1.1286 - acc: 0.8129
Epoch 7/100
1/1 [==============================] - 2s 2s/step - loss: 0.6803 - acc: 0.8830
Epoch 8/100
1/1 [==============================] - 2s 2s/step - loss: 0.4526 - acc: 0.8860
Epoch 9/100
1/1 [==============================] - 2s 2s/step - loss: 0.3251 - acc: 0.9327
Epoch 10/100
1/1 [==============================] - 2s 2s/step - loss: 0.2650 - ac

In [ ]:
# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.0013001967454329133 - Accuracy: 100.0%
------------------------------------------------------------------------
> Fold 2 - Loss: 0.10486336797475815 - Accuracy: 97.36841917037964%
------------------------------------------------------------------------
> Fold 3 - Loss: 0.007954839617013931 - Accuracy: 100.0%
------------------------------------------------------------------------
> Fold 4 - Loss: 8.091042400337756e-05 - Accuracy: 100.0%
------------------------------------------------------------------------
> Fold 5 - Loss: 0.0009619076736271381 - Accuracy: 100.0%
------------------------------------------------------------------------
> Fold 6 - Loss: 2.305675252500805e-06 - Accuracy: 100.0%
------------------------------------------------------------------------
> Fold 7 - Loss: 0.0019047793466597795 - Acc